# Quantum Integer Programming and Quantum Machine Learning 47-779/785, Tepper School of Business.
# Introduction to Quantum Computing 18-819F, Electrical and Computers Engineering.
# Fall 2021, Carnegie Mellon University

This notebook contains material from the Quantum Integer Programming and Quantum Machine Learning Course at CMU Fall 2021 by David Bernal (bernalde at cmu.edu), Sridhar Tayur (stayur at cmu.edu), Elias Towe (towe at cmu.edu), and Davide Venturelli; the content is available on **[Github](https://github.com/bernalde/QuIPML)**.
The text is released under the **[CC-BY-NC-ND-4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode) license, and code is released under the **[MIT license](https://opensource.org/licenses/MIT).*

This notebook makes simple computations of Graver basis. Because of the complexity of these computation, we suggest that for more complicated problems you install the excellent **[4ti2](https://4ti2.github.io/)** software, an open-source implementation of several routines useful for the study of integer programming through algebraic geometry. It can be used as a stand-alone library and call it from C++ or from Python. In python, there are two ways of accessing it, either through **[Sage](https://www.sagemath.org/)** (which is an open-source mathematics software) or directly compiling it and installing a thing **[Python wrapper](https://github.com/alfsan/Py4ti2)**

Run in **[Google Colab](https://colab.research.google.com/github/bernalde/QuIP/blob/master/notebooks/Notebook%203%20-%20Graver%20basis.ipynb)**

## Introduction to Graver basis computation
In this notebook, we will be covering the computation of Graver basis, which are defined as
$$
\mathcal{G}(\mathbf{A}) = \bigcup_j \mathcal{H}_j(\mathbf{A})
$$
where $\mathcal{H}_j(\mathbf{A})$ are the minimal Hilbert basis of $\mathbf{A}$ in each orthant.

The Graver basis relate to the Groebner basis we saw in the previous lecture in the sense that they contain the Universal Groebner basis (union over all the possible objective functions)
$$
\mathcal{U}(\mathbf{A}) = \bigcup_{c\in\mathbb{Z}^n} \mathcal{B}(\mathbf{A},\mathbf{c})  \subseteq \mathcal{G}(\mathbf{A})
$$

In [1]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools
import random

Let's first define certain functions that help us write down the Toric ideal of a matrix $\mathbf{A}$.

In [2]:
def to_polynomial(coef,vars):
    '''
    Function to define a single column of the coefficient as a polynomial
    '''
    res1 = 1
    res2 = 1
    for i in range(len(coef)):
        if coef[i] >= 0:
            res1 = res1*vars[i]**coef[i]
        else:
            res2 = res2*vars[i]**(-coef[i])
    res = res1 - res2
    return res

In [3]:
def polynomial_ideal(A):
    '''
    Function to define a the polynomial ideal of a matrix A according to Conti and Traverso
    '''
    IA = A.col_insert(0, eye(A.shape[0]))
    # Find nullspace (kernel) of A
    ker = IA.nullspace()

    # Normalize elements of kernel to be integers
    for i in range(len(ker)):
        rationalvector = True
        while rationalvector:
            factor = 1
            for j in ker[i]:
                if j%1 != 0:
                    factor = min(factor,j%1)
            if factor == 1:
                rationalvector = False
            else:
                ker[i]=ker[i] / factor
    
    # Define symbolic variables zs for each row (index 0 in Python)
    sym_str_z = 'z:' + str(A.shape[0])
    zs = symbols(sym_str_z)

    # Define symbolic variables ws for each column (index 0 in Python)
    sym_str_w = 'w:' + str(A.shape[1])
    ws = symbols(sym_str_w)

    vars = zs + ws

    sys = []
    for k in ker:
        sys.append(to_polynomial(k,vars))


    vars = ws + zs
    
    return(sys, vars)

First example, consider the same matrices as in the previous notebook
$$
4x_1 + 5x_2 + x_3 = 37 \\
2x_1 + 3x_2 + x_4 = 20 \\
\mathbf{x} \in \mathbb{Z}_+^4
$$


In [4]:
# A = np.array([[1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
#             [0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1],
#             [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]])
A = np.array([[4,5,1,0], [2,3,0,1]])

A = Matrix(A)

IA, vars = polynomial_ideal(A)
print(IA)
print(vars)
randvars = tuple(random.sample(vars, len(vars)))
result = groebner(IA, vars, order='lex')
print(result)

[w0 - z0**4*z1**2, w1 - z0**5*z1**3, w2 - z0, w3 - z1]
(w0, w1, w2, w3, z0, z1)
GroebnerBasis([w0 - z0**4*z1**2, w1 - z0**5*z1**3, w2 - z0, w3 - z1], w0, w1, w2, w3, z0, z1, domain='ZZ', order='lex')


In [5]:
# reduced(vars[-3]*vars[-2]*vars[-1],result)
reduced(vars[-2]**37*vars[-1]**20,result)

([-w0**8*z0*z1**2 - w0**7*z0**5*z1**4 - w0**6*z0**9*z1**6 - w0**5*z0**13*z1**8 - w0**4*z0**17*z1**10 - w0**3*z0**21*z1**12 - w0**2*z0**25*z1**14 - w0*z0**29*z1**16 - z0**33*z1**18,
  0,
  -w0**9*z1**2,
  -w0**9*w2*w3 - w0**9*w2*z1],
 w0**9*w2*w3**2)

In [6]:
def to_lawrence_polynomial(coef,xs,ys):
    '''
    Function to define a single column of the coefficient as a polynomial
    '''
    res1 = 1
    res2 = 1
    for i in range(len(coef)):
        if coef[i] >= 0:
            res1 = res1*xs[i]**coef[i]
            res2 = res2*ys[i]**coef[i]
        else:
            res2 = res2*xs[i]**(-coef[i])
            res1 = res1*ys[i]**(-coef[i])
    res = res1 - res2
    return res

In [7]:
def lawrence_polynomial_ideal(A):
    '''
    Function to define a the polynomial ideal of the Lawrence lifting of matrix A
    '''
    # Find nullspace (kernel) of A
    ker = A.nullspace()

    # Normalize elements of kernel to be integers
    for i in range(len(ker)):
        rationalvector = True
        while rationalvector:
            factor = 1
            for j in ker[i]:
                if j%1 != 0:
                    factor = min(factor,j%1)
            if factor == 1:
                rationalvector = False
            else:
                ker[i]=ker[i] / factor
    
    # Define symbolic variables zs for each row (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)

    # Define symbolic variables ws for each column (index 0 in Python)
    sym_str_y = 'y:' + str(A.shape[1])
    ys = symbols(sym_str_y)

    sys = []
    for k in ker:
        sys.append(to_lawrence_polynomial(k,xs,ys))
    
    return(sys, xs, ys)

In [8]:
LA, xs, ys = lawrence_polynomial_ideal(A)
print("Lawrence polynomial ideal")
print(LA)
randvars = tuple(random.sample(xs + ys, len(xs + ys)))
result_lawrence = groebner(LA, randvars, order='lex')
grav = []
for g in result_lawrence:
    for y in ys:
        g = g.subs({(y,1)})
    grav.append(g)

print("Reduced Graver (Groebner since we computed it with the Lawrence ideal) basis")
groebner(grav, xs, order='grevlex')

Lawrence polynomial ideal
[-x0**3*y1**2*y2**2 + x1**2*x2**2*y0**3, x0**5*x3**2*y1**4 - x1**4*y0**5*y3**2]
Reduced Graver (Groebner since we computed it with the Lawrence ideal) basis


GroebnerBasis([x0**5*x3**2 - x1**4, x1**2*x2**2 - x0**3], x0, x1, x2, x3, domain='ZZ', order='grevlex')

## Let's go back to the slides

In [9]:
# If using this on Google collab, if not we can import 4ti2
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's start with Pyomo
if not IN_COLAB:
    from Py4ti2int32 import *

In [10]:
# Define rules to choose augmentation element, either the best one (argmin) or the first one that is found
def argmin(iterable):
    return min(enumerate(iterable), key=lambda x: x[1])

def greedy(iterable):
    for i, val in enumerate(iterable):
        if val[1] != 0:
            return i, val
    else:
        return i, val

In [11]:
# Bisection rules for finding best step size
def bisection(g: np.ndarray, fun: Callable, x: np.ndarray, x_lo: np.ndarray = None, x_up: np.ndarray = None, laststep: np.ndarray = None) -> (float, int):
    if np.array_equal(g, laststep):
        return (fun(x), 0)
    if x_lo is None:
        x_lo = np.zeros_like(x)
    if x_up is None:
        x_up = np.ones_like(x)*max(x)*2

    u = max(x_up) - min(x_lo)
    l = -(max(x_up) - min(x_lo))
    for i, gi in enumerate(g):
        if gi >= 1:
            if np.floor((x_up[i] - x[i]) / gi) < u:
                u = int(np.floor((x_up[i] - x[i]) / gi))
            if np.ceil((x_lo[i] - x[i]) / gi) > l:
                l = int(np.ceil((x_lo[i] - x[i]) / gi))
        elif gi <= -1:
            if np.ceil((x_up[i] - x[i]) / gi) > l:
                l = int(np.ceil((x_up[i] - x[i]) / gi))
            if np.floor((x_lo[i] - x[i]) / gi) < u:
                u = int(np.floor((x_lo[i] - x[i]) / gi))
    alpha = u

    while u - l > 1:
        if fun(x + l*g) < fun(x + u*g):
            alpha = l
        else:
            alpha = u
        p1 = int(np.floor((l+u)/2) - 1)
        p2 = int(np.floor((l+u)/2))
        p3 = int(np.floor((l+u)/2) + 1)
        if fun(x + p1*g) < fun(x + p2*g):
            u = int(np.floor((l+u)/2))
        elif fun(x + p3*g) < fun(x + p2*g):
            l = int(np.floor((l+u)/2) + 1)
        else:
            alpha = p2
            break

    if fun(x + l*g) < fun(x + u*g) and fun(x + l*g) < fun(x + alpha*g):
        alpha = l
    elif fun(x + u*g) < fun(x + alpha*g):
        alpha = u

    return (fun(x + alpha*g), alpha)

In [12]:
# We can just have a single step move (works well with greedy approach)
def single_move(g: np.ndarray, fun: Callable, x: np.ndarray, x_lo: np.ndarray = None, x_up: np.ndarray = None, laststep: np.ndarray = None) -> (float, int):
    if x_lo is None:
        x_lo = np.zeros_like(x)
    if x_up is None:
        x_up = np.ones_like(x)*max(x)*2

    alpha = 0

    if (x + g <= x_up).all() and (x + g >= x_lo).all():
        if fun(x + g) < fun(x):
            alpha = 1
    elif (x - g <= x_up).all() and (x - g >= x_lo).all():
        if fun(x - g) < fun(x) and fun(x - g) < fun(x + g):
            alpha = -1
    
    return (fun(x + alpha*g), alpha)

In [13]:
EXAMPLE = 3

# Each example defines a problem min f(x)=cx s.t. Ax=b, x_lo <= x <= x_up, x integer

if EXAMPLE == 1:
    # Fist example - Illustrative example Alghassi et al. Graver Bases via Quantum Annealing
    # with application to non-linear integer programs
    A = np.array([[1, 1, 1, 1], [1, 5, 10, 25]])
    b = np.array([[21],[156]])
    # Objective is to minimize distance to 5
    x0 = np.array([1,15,3,2])
    x_lo = np.zeros_like(x0)
    x_up = 15*np.ones_like(x0)
elif EXAMPLE == 2:
    # Second example
    A = np.array([[1, 1, 1, 1], [1, 2, 3, 4]])
    b = np.array([[10], [21]])
    c = np.array([0, 1, 0, 2])
    x0 = np.array([1,8,0,1])
    x_lo = np.zeros_like(x0)
    x_up = 21*np.ones_like(x0)
elif EXAMPLE == 3:
    # Third example
    A = np.array([[1, 1, 1, 1], [0, 1, 2, 3]])
    b = np.array([[10],[15]])
    c = np.array([1, 3, 14, 17])
    x0 = np.array([3,0,6,1])
    x_lo = np.zeros_like(x0)
    x_up = 10*np.ones_like(x0)
elif EXAMPLE == 4:
    # Fourth example
    A = np.array([[1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,1,0,1,0],
    [1,1,1,1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,1,1,1,1,1],
    [0,1,0,0,0,1,0,1,0,1,1,0,1,1,0,1,1,0,0,1,0,0,1,1,1],
    [0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0],
    [0,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0]])
    b = np.array([[9], [8], [7], [5], [5]])
    # Objective is quadratic expression
    x0 = np.array([1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -2,
               1, 0, -1, 0, 1, -1, 1, -2, -2, 1, 1, 1])
    x_lo = np.zeros_like(x0)
    x_up = np.ones_like(x0)
else:
    print("Example not implemented")

if not IN_COLAB:
    r = graver("mat", A.tolist())
else:
    if EXAMPLE == 1:
        # Taken from paper but the same as computed with 4ti2
        r = [[5, -6, 0, 1], [5, -9, 4, 0], [0, 3, -4, 1], [5, -3, -4, 2], [5, 0, -8, 3]]
    elif EXAMPLE == 2:
        r = [[1, -2, 1, 0], [2, -3, 0, 1], [1, -1, -1, 1], [0, 1, -2, 1], [1, 0, -3, 2]]
    elif EXAMPLE == 3:
        r = [[2, -3, 0, 1], [1, -2, 1, 0], [1, -1, -1, 1], [0, 1, -2, 1], [1, 0, -3, 2]]
    elif EXAMPLE == 4:
        print("The Graver basis has 29789 elements in this case")
    else:
        print("Example not implemented")

r = np.array(r)

In [14]:
# Objective function definition
def f(x):
    if EXAMPLE == 1:
        return np.sum(np.abs(x - 5))
    elif EXAMPLE == 4:
        epsilon = 0.01
        mu = np.random.rand(len(x0))
        sigma = np.multiply(np.random.rand(len(x0)),mu)
        return -np.dot(mu, x) + np.sqrt(((1-epsilon)/epsilon)*np.dot(sigma**2, x**2))
    else:
        return np.dot(c,x)

# Constraints definition
def const(x):
    return np.array_equiv(np.dot(A,x),b.T) or np.array_equiv(np.dot(A,x),b)


In [15]:
# Let's perform the augmentation
OPTION = 1 # Best augmentation, select using bisection rule
# OPTION = 2 # Greedy augmentation, select using bisection rule
# OPTION = 3 # Greedy augmentation, select using first found

dist = 1
gprev = None
k = 1
print("Initial point:", x0)
while dist != 0:
    if OPTION == 1:
        g1, (obj, dist) = argmin(
            bisection(e, f, x0, laststep=gprev, x_lo=x_lo, x_up=x_up) for e in r)
    elif OPTION == 2:
        g1, (obj, dist) = greedy(
            bisection(e, f, x0, laststep=gprev, x_lo=x_lo, x_up=x_up) for e in r)
    elif OPTION == 3:    
        g1, (obj, dist) = greedy(
            single_move(e, f, x0, x_lo=x_lo, x_up=x_up) for e in r)
    else:
        print("Option not implemented")
        break
    x0 = x0 + r[g1]*dist
    gprev = r[g1]
    print("Iteration ", k)
    print(g1, (obj, dist))
    print("Augmentation direction:", gprev)
    print("Distanced moved:", dist)
    print("Step taken:", r[g1]*dist)
    print("Objective function:", obj)
    print("Current point:", x0)
    print("Are constraints satisfied?", const(x0))
    k += 1

Initial point: [3 0 6 1]
Iteration  1
1 (77, -3)
Augmentation direction: [ 1 -2  1  0]
Distanced moved: -3
Step taken: [-3  6 -3  0]
Objective function: 77
Current point: [0 6 3 1]
Are constraints satisfied? True
Iteration  2
3 (69, 1)
Augmentation direction: [ 0  1 -2  1]
Distanced moved: 1
Step taken: [ 0  1 -2  1]
Objective function: 69
Current point: [0 7 1 2]
Are constraints satisfied? True
Iteration  3
0 (69, 0)
Augmentation direction: [ 2 -3  0  1]
Distanced moved: 0
Step taken: [0 0 0 0]
Objective function: 69
Current point: [0 7 1 2]
Are constraints satisfied? True
